In [4]:
!pip install tenseal

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------------------- -------------------- 1.0/2.2 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 9.4 MB/s eta 0:00:00


In [2]:
import tenseal as ts

# Create a TenSEAL context with BFV scheme
context = ts.context(
    scheme=ts.SCHEME_TYPE.BFV,
    poly_modulus_degree=8192,
    plain_modulus=1032193,
    coeff_mod_bit_sizes=[60, 40, 40, 60],
)

# Generate secret key before making the context public
secret_key = context.secret_key()

# Enable encryption, decryption, and evaluation keys
context.generate_galois_keys()
context.generate_relin_keys()
context.make_context_public()  # Remove secret key for security

# Encrypt two numbers
enc_x = ts.bfv_vector(context, [3])
enc_y = ts.bfv_vector(context, [5])

# Perform encrypted operations
enc_add = enc_x + enc_y  # Homomorphic addition
enc_mul = enc_x * enc_y  # Homomorphic multiplication

# Decrypt results using the secret key
print("Decrypted Sum:", enc_add.decrypt(secret_key))  # Should output [8]
print("Decrypted Product:", enc_mul.decrypt(secret_key))  # Should output [15]


Decrypted Sum: [8]
Decrypted Product: [15]


In [5]:
# Encrypt two vectors
enc_vec1 = ts.bfv_vector(context, [1, 2, 3, 4, 5])
enc_vec2 = ts.bfv_vector(context, [10, 20, 30, 40, 50])

# Element-wise encrypted addition and multiplication
enc_sum = enc_vec1 + enc_vec2
enc_prod = enc_vec1 * enc_vec2

# Decrypt and print results
print("Decrypted Sum:", enc_sum.decrypt(secret_key))  
print("Decrypted Product:", enc_prod.decrypt(secret_key))  


Decrypted Sum: [11, 22, 33, 44, 55]
Decrypted Product: [10, 40, 90, 160, 250]


In [8]:
# Encrypt vectors
enc_vec1 = ts.bfv_vector(context, [1, 2, 3])
enc_vec2 = ts.bfv_vector(context, [4, 5, 6])

# Compute dot product (sum of element-wise multiplication)
enc_dot_product = (enc_vec1 * enc_vec2).sum()

# Decrypt and print result
print("Decrypted Dot Product:", enc_dot_product.decrypt(secret_key))  # Should output 32


Decrypted Dot Product: [32]


In [10]:
# Encrypt a dataset (e.g., salaries)
enc_data = ts.bfv_vector(context, [50000, 60000, 70000, 80000, 90000])

# Compute the encrypted sum
enc_sum = enc_data.sum()

# Compute encrypted mean (division is not natively supported in BFV, so we use plaintext division)
n = len(enc_data.decrypt(secret_key))  # Number of elements
mean = enc_sum.decrypt(secret_key)[0] / n

print("Decrypted Mean Salary:", mean)  # Should output 70000


Decrypted Mean Salary: 70000.0
